### Importing libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
import torchmetrics
import wandb

### Download Dataset

In [ ]:
# Mount Google Drive and unzip the dataset
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# Unzip dataset if not already unzipped
zip_path = '/content/drive/MyDrive/inaturalist_12K.zip'
extract_path = '/content/inaturalist_12K'

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/')


Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
import wandb
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm

# Define a dictionary to map activation function names to their corresponding PyTorch classes
activation_map = {
    'ReLU': nn.ReLU,
    'GELU': nn.GELU,
    'SiLU': nn.SiLU,
    'Mish': nn.Mish
}

# Flexible Convolutional Neural Network class definition
class FlexibleCNN(nn.Module):
    def __init__(self, conv_filters, kernel_sizes, activation_fn_str, dense_units, dropout_rate=0.5, num_classes=10):
        super(FlexibleCNN, self).__init__()

        # Validate activation function string
        if activation_fn_str not in activation_map:
            raise ValueError(f"Unsupported activation function: {activation_fn_str}")
        activation_fn = activation_map[activation_fn_str]

        self.convs = nn.ModuleList()
        in_channels = 3  # RGB channels for images

        # Create convolutional blocks with Conv2D -> Activation -> MaxPooling
        for filters, k in zip(conv_filters, kernel_sizes):
            self.convs.append(nn.Sequential(
                nn.Conv2d(in_channels, filters, kernel_size=k, padding=k // 2),
                activation_fn(),  # Activation function instance
                nn.MaxPool2d(kernel_size=2)
            ))
            in_channels = filters  # Update for next layer

        self.flatten = nn.Flatten()

        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(dense_units, num_classes)

    def forward(self, x):
        for conv in self.convs:
            x = conv(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Utility function to calculate accuracy
def accuracy(preds, labels):
    _, pred_classes = preds.max(1)
    correct = (pred_classes == labels).sum().item()
    return correct / len(labels)

# Load training and validation data using stratified split
def load_data(data_dir, batch_size, val_split=0.2):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    full_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
    targets = [label for _, label in full_dataset.imgs]

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=val_split, random_state=42)
    train_idx, val_idx = next(splitter.split(full_dataset.imgs, targets))

    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

# Load test data
def load_test_data(data_dir, batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_loader, test_dataset

# Main training function integrated with Weights & Biases
def train_model(config=None):
    wandb.init(config=config)
    config = wandb.config

    # Apply data augmentation only if specified
    if config.data_augmentation == 'yes':
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # Select device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize model
    model = FlexibleCNN(
        conv_filters=config.conv_filters,
        kernel_sizes=config.kernel_sizes,
        activation_fn_str=config.activation_fn,
        dense_units=config.dense_units,
        dropout_rate=config.dropout,
        num_classes=10
    ).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

    # Load data loaders
    train_loader, val_loader = load_data(config.data_dir, config.batch_size)

    # Training loop
    for epoch in range(config.epochs):
        model.train()
        total_train_loss, total_train_acc = 0, 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
            inputs, labels = batch[0].to(device), batch[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            total_train_acc += accuracy(outputs, labels)

        # Validation loop
        model.eval()
        total_val_loss, total_val_acc = 0, 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
                inputs, labels = batch[0].to(device), batch[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                total_val_acc += accuracy(outputs, labels)

        # Compute average metrics
        train_acc = total_train_acc / len(train_loader)
        val_acc = total_val_acc / len(val_loader)

        # Log metrics to Weights & Biases
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': total_train_loss / len(train_loader),
            'train_accuracy': train_acc,
            'val_loss': total_val_loss / len(val_loader),
            'val_accuracy': val_acc
        })

# Sweep Configuration 
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'conv_filters': {
            'values': [[32]*5, [32, 64, 128, 128, 256], [64]*5]
        },
        'kernel_sizes': {
            'values': [[3]*5, [5]*5]
        },
        'activation_fn': {
            'values': ['ReLU', 'GELU', 'SiLU']
        },
        'dense_units': {
            'values': [128, 256, 512]
        },
        'dropout': {
            'values': [0.0, 0.3, 0.5]
        },
        'lr': {
            'max': 0.01,
            'min': 0.0001
        },
        'batch_size': {
            'values': [32, 64]
        },
        'epochs': {
            'value': 10
        },
        'data_dir': {
            'value': '/content/inaturalist_12K'
        }
    }
}

# To run the sweep
sweep_id = wandb.sweep(sweep_config, project="A2")
wandb.agent(sweep_id, function=train_model, count=20)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: szhxr0m3
Sweep URL: https://wandb.ai/me21b164-indian-institute-of-technology-madras/A2/sweeps/szhxr0m3


wandb: Agent Starting Run: n3lyoluh with config:
wandb: 	activation_fn: GELU
wandb: 	batch_size: 32
wandb: 	conv_filters: [32, 32, 32, 32, 32]
wandb: 	data_dir: /content/inaturalist_12K
wandb: 	dense_units: 128
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	kernel_sizes: [5, 5, 5, 5, 5]
wandb: 	lr: 0.005911937250248187
wandb: Currently logged in as: me21b164 (me21b164-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 10 [Val]: 100%|██████████| 32/32 [00:09<00:00,  3.48it/s]


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▂▆█▅▄▄▇▄
train_loss,▂▁█▁▁▁▁▁▁▁
val_accuracy,▇▅▅▁▁▁▅▁█▅
val_loss,▁█▂▅▄▂▃▂▂▃
epoch,10
train_accuracy,0.09782
train_loss,2.30682
val_accuracy,0.10059
val_loss,2.30403


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: l7pxfaty with config:
wandb: 	activation_fn: GELU
wandb: 	batch_size: 32
wandb: 	conv_filters: [64, 64, 64, 64, 64]
wandb: 	data_dir: /content/inaturalist_12K
wandb: 	dense_units: 256
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.00048541512186049175


Epoch 10 [Val]: 100%|██████████| 32/32 [00:08<00:00,  3.90it/s]


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▃▄▅▆▆▇██
train_loss,█▇▆▆▅▄▃▃▂▁
val_accuracy,▁▃▅▅█▇█▇█▇
val_loss,█▆▅▄▂▂▁▁▁▁
epoch,10
train_accuracy,0.43136
train_loss,1.60969
val_accuracy,0.33105
val_loss,1.87049


wandb: Agent Starting Run: pjv0ad6g with config:
wandb: 	activation_fn: SiLU
wandb: 	batch_size: 64
wandb: 	conv_filters: [64, 64, 64, 64, 64]
wandb: 	data_dir: /content/inaturalist_12K
wandb: 	dense_units: 512
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.0006455187673613949


Epoch 9 [Train]:  83%|████████▎ | 52/63 [00:34<00:06,  1.59it/s]

### **Second Session**

Choosing selected hyperparameters for longer runs

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'activation_fn': {
            'values': ['GELU', 'SiLU']
        },
        'batch_size': {
            'values': [32, 64]
        },
        'conv_filters': {
            'values': [[64, 64, 64, 64, 64]]
        },
        'kernel_sizes': {
            'values': [[3, 3, 3, 3, 3]]
        },
        'dense_units': {
            'values': [128, 256, 512]
        },
        'dropout': {
            'values': [0.3, 0.5]
        },
        'lr': {
            'distribution': 'uniform',
            'min': 0.0003,
            'max': 0.0007
        },
        'epochs': {
            'value': 20
        },
        'data_dir': {
            'value': '/content/inaturalist_12K'
        },
        'data_augmentation': {
            'values': ['yes', 'no']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="A2")
wandb.agent(sweep_id, function=train_model, count=20)